# Create Time-Lapse Video at Noon

Before we can create the time-lapse video, we must select images which should be present in the video. We already did this, using only one image for each day of the year, around 12am and 1pm, see [image-selection-noon](./1.0-image-selection-noon.ipynb) notebook.

The result of the image selection process, was a list of images potentially selected for the time-lapse video. Why potentially? Because we do not know whether the selected images correspond to the *open* or *closed* class, yet. For this we use our already trained *logistic-regression* model to determine the class of each image. Once we classified each image, we can create our time-lapse video, considering only images related to the *open* class.

We load our pre-trained model from the filesystem and predict the class for each image in the image directory given to the model. Features must not be extracted manually, this will automatically be done by the special prediction method call.

In [24]:
from src.models import predict_model

IMG_DIR = '../data/interim/selected-images-noon'
MODEL_PATH = "../models/logreg-clf-notebook.joblib"
CLOSED_CLASS = 1

clf = predict_model.load_model(MODEL_PATH)
result = predict_model.extract_features_predict(IMG_DIR, clf)

# how many images were detected as 'closed'
closed_total = result['prediction'].sum()
print("Number of 'closed' images: {} out of {}".format(closed_total, result.shape[0]))

# display all rows where predicted class was 'closed'
print("Paths of 'closed' files")
closed_file_paths = result[result['prediction'] == CLOSED_CLASS]['file']
print(closed_file_paths)

Number of 'closed' images: 26 out of 363
Paths of 'closed' files
7      ../data/interim/selected-images-noon/pic_14936...
14     ../data/interim/selected-images-noon/pic_14954...
19     ../data/interim/selected-images-noon/pic_14967...
34     ../data/interim/selected-images-noon/pic_15021...
42     ../data/interim/selected-images-noon/pic_15050...
50     ../data/interim/selected-images-noon/pic_15082...
52     ../data/interim/selected-images-noon/pic_15089...
54     ../data/interim/selected-images-noon/pic_15091...
69     ../data/interim/selected-images-noon/pic_15166...
90     ../data/interim/selected-images-noon/pic_15027...
99     ../data/interim/selected-images-noon/pic_15004...
110    ../data/interim/selected-images-noon/pic_15012...
111    ../data/interim/selected-images-noon/pic_15003...
114    ../data/interim/selected-images-noon/pic_15011...
116    ../data/interim/selected-images-noon/pic_14937...
142    ../data/interim/selected-images-noon/pic_15096...
162    ../data/interim/

Our model determined **26** out of **363** images to be in the *closed* class. Some will notice that we only have selected 363 images, instead of having 365 to span an entire year. It is possible that there were no images for some days in the given timeframe, *noon +/- 5 minutes*, and therefore, not every day has been considered  in the selection process.

For the time-lapse video, select images of class *open*, only. Therefore, we save the file paths identified to be member of the *open* class into a text file. This text file will later be used as list of selected images for video creation on the linux command line.

In [15]:
# write image paths to file, which were identified to be member in *open* class
OPEN_IMAGES_FILE = '../data/processed/file-paths-noon-open.txt'
OPEN_CLASS = 0

open_file_paths = result[result['prediction'] == OPEN_CLASS]['file']

open_file_paths.to_csv(OPEN_IMAGES_FILE, header = False, index = False)

## Create Time-Lapse video

Use *ffmpeg* to create the time-lapse video. The next command consists of multiple steps, *piped* or *chained* together to form a *single* command, which does the following

1. read each file path from file we just created, using *cat*
1. search for filenames, using regex pattern, *grep*
1. sort filenames, to get them in ascending order
1. use *xargs* to execute *find* for each filename
1. use *find* to get file path, exec *cat* to get *bytes* of each file
1. use *ffmpeg* to create time-lapse video, incl. timestamp as watermark, which is read from images' EXIF metadata


```
$ cat ../data/processed/file-paths-noon-open.txt | \
    grep -oE "pic_[0-9]{10}.jpg" | \
    sort | \
    xargs -I# find ../data/interim/selected-images-noon -name "*#*" -exec cat {} \; | \
    ffmpeg -r 24 -f image2pipe -i - -filter_complex "drawtext=fontfile=/Windows/Fonts/arial.ttf:text='timestamp \: %{metadata\:DateTime}':x=5:y=5:fontsize=24:fontcolor=white@0.9:box=1:boxcolor=blue@0.6" -s 1024x768 -vcodec libx264 time-lapse-noon.mp4
```

In [23]:
import os

COMMAND = 'cat {filenames} | \
    grep -oE "pic_[0-9]{{10}}.jpg" | \
    sort | \
    xargs -I# find {image_dir} -name "*#*" -exec cat {{}} \; | \
    ffmpeg -r 24 -f image2pipe -i - -filter_complex \
        "drawtext=fontfile={font_file}:text=\'timestamp \: %{{metadata\:DateTime}}\':x=5:y=5:fontsize=24:fontcolor=white@0.9:box=1:boxcolor=blue@0.6" \
        -s 1024x768 -vcodec libx264 {video_file_name}'

IMAGE_DIR = '../data/interim/selected-images-noon/' # trailing slash required, due to symlinking
VIDEO_FILE_NAME = '../data/processed/time-lapse-noon.mp4'
FONT_FILE = '/usr/share/fonts/TTF/DejaVuSans.ttf'

# delete existing video, otherwise command will fail
if os.path.exists(VIDEO_FILE_NAME):
  os.remove(VIDEO_FILE_NAME)

# execute command
os.system(COMMAND.format(
    filenames = OPEN_IMAGES_FILE,
    image_dir = IMAGE_DIR,
    video_file_name = VIDEO_FILE_NAME,
    font_file = FONT_FILE) )

0

## Result

With a frame rate set to *24* images per second, we get a video of 14 seconds. It's quiet nice to see the vegetation in all four seasons.

If we take a close look, we can determine that in a single frame there was a *closed shutter* image. This means that our trained logistic-regression classifier was not able to detect this image correctly, and we have one *false negative* in this case. The rest of the time-lapse video looks ok-ish, at least considering correct image classification, only. However, there are some frames showing clearly visible reflections and we may choose another time frame to reduce reflections, e.g. choose some time in the early afternoon.

<!-- Simple video example -->
<video src="../data/processed/time-lapse-noon.mp4" autoplay width=800px loop=true type="video/mp4" controls>
  Sorry, your browser doesn't support embedded videos, 
  but don't worry, you can <a href="../data/processed/time-lapse-noon.mp4">download it</a>
  and watch it with your favorite video player!
</video>

[external link](https://drive.google.com/file/d/1y6d_VMWg0dm1nIgvE1iljanSD3jEP345/preview)